In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import time

import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

<IPython.core.display.Javascript object>

In [26]:
torch.manual_seed(0)

<IPython.core.display.Javascript object>

## Set up the neural network

Create a class for network structure.

Create helper functions to initialize the weights and to evaluate the training and testing functions.

In [4]:
class My_Net1(nn.Module):
    def __init__(self, n_features, n_classes, hidden_dim1=15, hidden_dim2=15):

        super(My_Net1, self).__init__()

        self.layer_1 = nn.Linear(n_features, hidden_dim1)
        self.layer_2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.layer_3 = nn.Linear(hidden_dim2, n_classes)

        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_data):
        out = self.layer_1(input_data)
        out = self.relu(out)
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.layer_3(out)
        out = self.softmax(out)
        return out

<IPython.core.display.Javascript object>

In [5]:
class My_Net2(nn.Module):
    def __init__(self, n_features, n_classes, hidden_dim1=15, hidden_dim2=15):

        super(My_Net2, self).__init__()

        self.layer_1 = nn.Linear(n_features, hidden_dim1)
        self.layer_2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.layer_3 = nn.Linear(hidden_dim2, n_classes)

        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_data):
        out = self.layer_1(input_data)
        out = self.sigmoid(out)
        out = self.layer_2(out)
        out = self.sigmoid(out)
        out = self.layer_3(out)
        out = self.softmax(out)
        return out

<IPython.core.display.Javascript object>

In [15]:
class My_Net3(nn.Module):
    def __init__(self, n_features, n_classes, hidden_dim1=15, hidden_dim2=15, hidden_dim3=15):

        super(My_Net3, self).__init__()

        self.layer_1 = nn.Linear(n_features, hidden_dim1)
        self.layer_2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.layer_3 = nn.Linear(hidden_dim2, n_classes)

        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_data):
        out = self.layer_1(input_data)
        out = self.tanh(out)
        out = self.layer_2(out)
        out = self.tanh(out)
        out = self.layer_3(out)
        out = self.softmax(out)
        return out

<IPython.core.display.Javascript object>

In [16]:
def weights_init(m):
    """Initialize the network using Xavier initialization."""
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight.data)

<IPython.core.display.Javascript object>

In [17]:
def train_eval(net, train_loader, loss, verbose=True):

    correct = 0
    total = 0
    loss_sum = 0
    num_batches = 0
    for inputs, labels in train_loader:
        outputs = net(inputs)
        predicted = torch.argmax(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted.int() == labels.int()).sum()
        loss_sum += loss(outputs, labels).item()
        num_batches += 1

    if verbose:
        print(f"Train accuracy: {100 * correct.item() / total:.3f}%")

    return loss_sum / num_batches, correct.item() / total

<IPython.core.display.Javascript object>

In [18]:
def test_eval(net, test_loader, loss, verbose=True):

    correct = 0
    total = 0
    loss_sum = 0
    num_batches = 0
    for inputs, labels in test_loader:
        outputs = net(inputs)
        predicted = torch.argmax(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted.int() == labels.int()).sum()
        loss_sum += loss(outputs, labels).item()
        num_batches += 1

    if verbose:
        print(f"Test accuracy: {100 * correct.item() / total:.3f}%")

    return loss_sum / num_batches, correct.item() / total

<IPython.core.display.Javascript object>

In [19]:
def main(train_loader, test_loader, net_initializer, n_features, n_classes, lr=1e-2, print_every=5, epochs=25):
    

    # Initialize the neural network
    net = net_initializer(n_features, n_classes)
    loss = nn.CrossEntropyLoss()
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    net.apply(weights_init)

    train_loss_store = []
    train_acc_store = []
    test_loss_store = []
    test_acc_store = []

    for epoch in range(epochs):
        verbose = False
        if (epoch == 0) or ((epoch + 1) % print_every) == 0:
            verbose = True
        
        time1 = time.time()
        
        if verbose:
            print(f"Epoch {epoch + 1}:")

        for i, (feat, label) in enumerate(train_loader, 0):
            opt.zero_grad()
            outputs = net(feat)
            l = loss(outputs, label)
            l.backward()
            opt.step()
    
        l_temp, acc_temp = train_eval(net, train_loader, loss, verbose=verbose)
        train_loss_store.append(l_temp)
        train_acc_store.append(acc_temp)

        l_temp, acc_temp = test_eval(net, test_loader, loss, verbose=verbose)
        test_loss_store.append(l_temp)
        test_acc_store.append(acc_temp)

        time2 = time.time()
        
        if verbose:
            print(f"Time lapse: {time2-time1:.2f} sec \n")

<IPython.core.display.Javascript object>

## Make a helper function to parse the data file

In [28]:
def parse_data(fpath):

    # Input the data and split into features and labels
    data = np.genfromtxt(fpath, dtype=np.int64, skip_header=7)
    X, y = data[:, :-1], data[:, -1]
    n_classes = len(np.unique(y))
    n_features = X.shape[1]

    # Index classes starting at 0 for CrossEntropyLoss() to work
    y -= 1

    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    # Convert to tensors
    X_train = torch.Tensor(X_train)
    X_test = torch.Tensor(X_test)
    y_train = torch.LongTensor(y_train)
    y_test = torch.LongTensor(y_test)

    # Pick optimal batch size
    if "stable4" in fpath:
        batch_size = 1
    elif "stable5" in fpath:
        batch_size = 32
    elif "stable6" in fpath:
        batch_size = 64

    # Make data loaders for the test and training set
    train = torch.utils.data.TensorDataset(X_train, y_train)
    test = torch.utils.data.TensorDataset(X_test, y_test)

    train_loader = torch.utils.data.DataLoader(
        train, batch_size=batch_size, num_workers=2
    )
    test_loader = torch.utils.data.DataLoader(
        test, batch_size=batch_size, shuffle=False, num_workers=2
    )

    return train_loader, test_loader, n_features, n_classes

<IPython.core.display.Javascript object>

## Train the model for each dataset

In [30]:
fpath = "stable4.txt"
train_loader, test_loader, n_features, n_classes = parse_data(fpath)
print("My_Net1")
print("--------")
main(train_loader, test_loader, My_Net1, n_features, n_classes, lr=5e-3)
print("My_Net2")
print("--------")
main(train_loader, test_loader, My_Net2, n_features, n_classes, lr=5e-3)
print("My_Net3")
print("--------")
main(train_loader, test_loader, My_Net3, n_features, n_classes, lr=5e-3)

My_Net1
--------
Epoch 1:
Train accuracy: 56.818%
Test accuracy: 80.000%
Time lapse: 0.21 sec 

Epoch 5:
Train accuracy: 79.545%
Test accuracy: 75.000%
Time lapse: 0.19 sec 

Epoch 10:
Train accuracy: 93.182%
Test accuracy: 85.000%
Time lapse: 0.20 sec 

Epoch 15:
Train accuracy: 95.455%
Test accuracy: 90.000%
Time lapse: 0.20 sec 

Epoch 20:
Train accuracy: 95.455%
Test accuracy: 85.000%
Time lapse: 0.19 sec 

Epoch 25:
Train accuracy: 95.455%
Test accuracy: 90.000%
Time lapse: 0.21 sec 

My_Net2
--------
Epoch 1:
Train accuracy: 54.545%
Test accuracy: 80.000%
Time lapse: 0.19 sec 

Epoch 5:
Train accuracy: 54.545%
Test accuracy: 80.000%
Time lapse: 0.20 sec 

Epoch 10:
Train accuracy: 63.636%
Test accuracy: 80.000%
Time lapse: 0.20 sec 

Epoch 15:
Train accuracy: 79.545%
Test accuracy: 80.000%
Time lapse: 0.20 sec 

Epoch 20:
Train accuracy: 86.364%
Test accuracy: 85.000%
Time lapse: 0.19 sec 

Epoch 25:
Train accuracy: 86.364%
Test accuracy: 85.000%
Time lapse: 0.19 sec 

My_Net3
--

<IPython.core.display.Javascript object>

In [31]:
fpath = "stable5.txt"
train_loader, test_loader, n_features, n_classes = parse_data(fpath)
print("My_Net1")
print("--------")
main(train_loader, test_loader, My_Net1, n_features, n_classes, lr=5e-3)
print("My_Net2")
print("--------")
main(train_loader, test_loader, My_Net2, n_features, n_classes, lr=5e-3)
print("My_Net3")
print("--------")
main(train_loader, test_loader, My_Net3, n_features, n_classes, lr=5e-3)

My_Net1
--------
Epoch 1:
Train accuracy: 37.989%
Test accuracy: 37.987%
Time lapse: 0.18 sec 

Epoch 5:
Train accuracy: 75.419%
Test accuracy: 73.701%
Time lapse: 0.15 sec 

Epoch 10:
Train accuracy: 79.330%
Test accuracy: 80.519%
Time lapse: 0.16 sec 

Epoch 15:
Train accuracy: 79.330%
Test accuracy: 82.468%
Time lapse: 0.15 sec 

Epoch 20:
Train accuracy: 83.101%
Test accuracy: 84.416%
Time lapse: 0.15 sec 

Epoch 25:
Train accuracy: 84.777%
Test accuracy: 85.390%
Time lapse: 0.16 sec 

My_Net2
--------
Epoch 1:
Train accuracy: 55.028%
Test accuracy: 57.143%
Time lapse: 0.15 sec 

Epoch 5:
Train accuracy: 55.028%
Test accuracy: 57.143%
Time lapse: 0.15 sec 

Epoch 10:
Train accuracy: 55.028%
Test accuracy: 57.143%
Time lapse: 0.16 sec 

Epoch 15:
Train accuracy: 75.559%
Test accuracy: 77.273%
Time lapse: 0.15 sec 

Epoch 20:
Train accuracy: 77.095%
Test accuracy: 78.896%
Time lapse: 0.15 sec 

Epoch 25:
Train accuracy: 77.095%
Test accuracy: 78.896%
Time lapse: 0.15 sec 

My_Net3
--

<IPython.core.display.Javascript object>

In [35]:
fpath = "stable6.txt"
train_loader, test_loader, n_features, n_classes = parse_data(fpath)
print("My_Net1")
print("--------")
main(train_loader, test_loader, My_Net1, n_features, n_classes, lr=5e-3)
print("My_Net2")
print("--------")
main(train_loader, test_loader, My_Net2, n_features, n_classes, lr=5e-3)
print("My_Net3")
print("--------")
main(train_loader, test_loader, My_Net3, n_features, n_classes, lr=5e-3)

My_Net1
--------
Epoch 1:
Train accuracy: 67.611%
Test accuracy: 67.104%
Time lapse: 0.95 sec 

Epoch 5:
Train accuracy: 73.789%
Test accuracy: 72.851%
Time lapse: 0.95 sec 

Epoch 10:
Train accuracy: 74.317%
Test accuracy: 73.136%
Time lapse: 0.95 sec 

Epoch 15:
Train accuracy: 75.559%
Test accuracy: 74.235%
Time lapse: 0.95 sec 

Epoch 20:
Train accuracy: 76.252%
Test accuracy: 74.957%
Time lapse: 0.96 sec 

Epoch 25:
Train accuracy: 76.719%
Test accuracy: 75.384%
Time lapse: 0.97 sec 

My_Net2
--------
Epoch 1:
Train accuracy: 66.713%
Test accuracy: 66.473%
Time lapse: 0.95 sec 

Epoch 5:
Train accuracy: 66.713%
Test accuracy: 66.473%
Time lapse: 0.95 sec 

Epoch 10:
Train accuracy: 66.713%
Test accuracy: 66.473%
Time lapse: 0.94 sec 

Epoch 15:
Train accuracy: 66.713%
Test accuracy: 66.473%
Time lapse: 0.95 sec 

Epoch 20:
Train accuracy: 66.713%
Test accuracy: 66.473%
Time lapse: 0.97 sec 

Epoch 25:
Train accuracy: 66.713%
Test accuracy: 66.473%
Time lapse: 0.96 sec 

My_Net3
--

<IPython.core.display.Javascript object>